> #### Version:V.1.1 
> #### Git:https://github.com/Zhuang-Zhuang-Liu/Titan-Analysis/tree/main
> #### Power_By:DeepSeek-Chat-V2
> #### Author:Zhuang-Zhuang-Liu
> #### Update_Date:20240623

In [ ]:
!pip install openai==1.35.3
!pip install pyautogen==0.2.31
!pip install 'pyautogen[jupyter-executor]'
!pip install "pyautogen[retrievechat]"

In [ ]:
import shutil
import sys
import json
import os

# git clone
current_directory = os.getcwd()
def clean_library_folder(library_path): 
    try:shutil.rmtree(library_path)
    except OSError as e:print(f"delete {library_path} error: {e}")
clean_library_folder(library_path = current_directory +'/Titan-Analysis')
!git clone https://github.com/Zhuang-Zhuang-Liu/Titan-Analysis.git
sys.path.append( current_directory +'/Titan-Analysis') 

# Titan-Analysis
from utils.utils import show_images_in_directory,folder_clean 
from utils.utils import generate_guide  # rag
from utils.utils import generate_random_data,data_info_put 
from agent_zoo.GroupChat import agent_create,titan_analysis,titan_load

In [ ]:
# llm api
api_zz_deep,url_deep,model_deep = 'sk-1cda68c98daf46a1b931687e698e9ca8','https://api.deepseek.com/v1',"deepseek-chat"
llm_config_deep = {"config_list": [{ "model": model_deep,"base_url": url_deep,"api_key": api_zz_deep,"temperature": 0,"cache_seed":None  }] }  

# guide rag
guide_path = current_directory +'/Titan-Analysis/rag_zoo/data_analysis_guide.txt'

# agent prompt
prompt_path = current_directory+'/Titan-Analysis/agent_zoo/agent_prompts.json'
with open(prompt_path, "r") as file: agent_prompts = json.load(file)

#folder path
path="coding"
folder_clean(path)

# load data and task
task_info,data_info = titan_load()

# analysis
titan_analysis(path=path,llm_config = llm_config_deep, loaded_data=agent_prompts, 
               guide_path=guide_path,
               task_info=task_info,data_info=data_info)